In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.metrics import (roc_auc_score, roc_curve, auc, accuracy_score, classification_report, 
confusion_matrix, accuracy_score)

from sklearn.model_selection import (cross_val_score, cross_val_predict, train_test_split, 
StratifiedKFold, GridSearchCV)

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.pipeline import Pipeline

from textblob import TextBlob, Word

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [2]:
'''
In this model, I have created dummified categorical variables of each key skill as identified in Question 2, part 3.
I then modelled the salary on these variables in order to train the model.
'''


df = pd.read_csv('./seek3.csv', index_col = 0)

In [3]:
df.drop(['job_location', 'url', 'job_post_date', 'job_salary', 'salary_desc', 'super', 'payt2'], axis=1, inplace=True)

In [4]:
df.search_category.value_counts()

Data Analyst             806
Data Science             161
Data Engineer            136
Business Intelligence    114
Name: search_category, dtype: int64

In [5]:
df = df[df.search_category == 'Data Analyst']

In [6]:
df.reset_index(drop = True, inplace=True)

In [7]:
df

,job_title,advertiser,search_category,job_desc,pay_total2
0,SQL Data Analyst (Junior/Mid Level) :: $62K,Correlate Resources,Data Analyst,SQL Data Analyst (Fixed Term Contract) :: Lead...,68200.00
1,Data Reporting Analyst - Finance Systems - Ban...,Bluefin Resources Pty Limited,Data Analyst,About youYou will provide high-level first lin...,100000.00
2,Data Analyst - Asset Management - 2 month cont...,Anton Murray Consulting,Data Analyst,"Position Description: Data gathering, analysis...",164250.00
3,Business Analyst - Data,2XM Technology Pty Ltd,Data Analyst,Develop complex modelling for the digital bus...,210787.50
4,Peoplesoft Functional Test Analyst,Hydrogen Group Pty Ltd,Data Analyst,The Peoplesoft Functional Test Analyst role wi...,202575.00
5,Data Analyst,Robert Half Technology,Data Analyst,List key benefits: (no more than 50 characters...,100000.00
6,Business / Data Analyst,Charterhouse,Data Analyst,Our NSW Government client are currently seekin...,183040.00
7,Research Analyst,Screen Australia,Data Analyst,We currently have an opportunity for a data dr...,92542.45
8,Business and Data Analyst,IRESS Limited,Data Analyst,About IRESS We began in 1993 as a small Austra...,119999.50
9,IT Business Analyst,face2face Recruitment,Data Analyst,One of our Federal Government Clients is seeki...,187200.00


In [8]:
df.pay_total2.median()

129999.75

In [10]:
df.pay_total2.value_counts()

110000.00    25
100000.00    15
301125.00    15
99000.00     13
115500.00    13
82500.00     13
137500.00    12
132000.00    12
321200.00    12
292000.00    11
120000.00    11
121000.00    11
150000.00    11
125000.00    11
281050.00    11
148500.00     9
126500.00     9
104500.00     9
115000.00     8
143000.00     8
165000.00     8
95000.00      8
88000.00      8
361350.00     8
273750.00     8
331237.50     8
114400.00     8
93500.00      8
130000.00     7
90000.00      7
             ..
350911.00     1
109999.50     1
54999.50      1
71000.00      1
197600.00     1
223836.25     1
231000.00     1
52250.00      1
63249.45      1
80850.00      1
59488.00      1
63733.00      1
124907.75     1
248200.00     1
291087.50     1
130487.50     1
113150.00     1
97900.00      1
158592.50     1
82368.00      1
162425.00     1
145600.00     1
140400.00     1
140250.00     1
57500.00      1
246375.00     1
138875.00     1
200500.00     1
104104.00     1
183040.00     1
Name: pay_total2, Length

In [11]:
df.job_desc = df.job_desc.str.lower()

In [12]:
df['agile'] = 0
df['cloud'] = 0
df['sql'] = 0
df['python'] = 0
df['excel'] = 0
df['aws'] = 0
df['sum'] = 0

In [13]:
df.loc[df.job_desc.str.contains('agile'), 'agile'] = 1
df.loc[df.job_desc.str.contains('cloud'), 'cloud'] = 1
df.loc[df.job_desc.str.contains('sql'), 'sql'] = 1
df.loc[df.job_desc.str.contains('python'), 'python'] = 1
df.loc[df.job_desc.str.contains('excel'), 'excel'] = 1
df.loc[df.job_desc.str.contains('aws'), 'aws'] = 1

In [14]:
df['sum'] = df['agile'] + df['cloud'] + df['sql'] + df['python'] + df['excel'] + df['aws']

In [15]:
df['sum'].value_counts()

1    374
0    253
2    123
3     45
4      9
5      2
Name: sum, dtype: int64

In [16]:
df = df[df['sum'] != 0]

In [17]:
df.pay_total2.median()

126500.0

In [18]:
df.pay_total2.value_counts()

110000.00    18
281050.00    10
121000.00    10
132000.00    10
301125.00    10
125000.00    10
100000.00    10
321200.00     9
99000.00      8
88000.00      8
82500.00      8
104500.00     8
331237.50     7
143000.00     7
126500.00     7
95000.00      7
137500.00     7
93500.00      6
148500.00     6
140000.00     6
115000.00     6
120000.00     6
255500.00     6
200750.00     6
165000.00     6
114400.00     6
115500.00     5
328500.00     5
150000.00     5
220825.00     5
             ..
80850.00      1
187000.00     1
293095.00     1
114999.50     1
52250.00      1
200750.00     1
170500.00     1
52500.00      1
62499.50      1
264990.00     1
150562.50     1
233600.00     1
104025.00     1
131400.00     1
63733.00      1
93600.00      1
219000.00     1
91850.00      1
315725.00     1
55000.00      1
401500.00     1
54080.00      1
129999.50     1
99999.50      1
85800.00      1
92000.00      1
294701.00     1
132500.00     1
341475.75     1
183040.00     1
Name: pay_total2, Length

In [32]:
7/236

0.029661016949152543

In [20]:
X = df[['agile', 'cloud', 'sql', 'python', 'excel', 'aws']].copy()

In [21]:
y = df['pay_total2']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [23]:

model2 = Pipeline(
            steps = [
                    ('rfr', RandomForestRegressor(n_estimators=370,
                                                  max_depth=45,
                                                  random_state=42))]
                      )

model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
    
print(model2.score(X_test,y_test))
#     print("Number of features:", len(model2.steps[0][1].get_feature_names()))

    # print(model.steps[0][1].get_feature_names())
print(model2.steps[0][1].feature_importances_)
    # print(model.named_steps['rfc'].feature_importances_)



#     feature_importances = pd.DataFrame(model2.steps[1][1].feature_importances_, model2.steps[0][1].get_feature_names()).reset_index()
#     feature_importances.columns = ['feature', 'importance']
#     features = feature_importances.sort_values('importance', ascending=False)
#     print(features.sort_values('importance', ascending=False).head(60))

0.13696370808522162
[0.47868312 0.07699023 0.12882664 0.07869702 0.13538056 0.10142242]


In [ ]:
model2 = Pipeline(
            steps = [ 
                      ('gs', GridSearchCV(RandomForestRegressor(),
                                          param_grid={'n_estimators': [365, 370, 375],
                                                      'max_depth' : [40, 45, 50]},
                                                      cv=5,
                                                      refit=True))]
                  )

model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

print(model2.score(X_test,y_test))
# print("Number of features:", len(model2.steps[0][1].get_feature_names()))

# print(model.steps[0][1].get_feature_names())
# print(model.steps[1][1].feature_importances_)
# print(model.named_steps['rfc'].feature_importances_)


print(model2.steps[0][1].best_params_)



In [24]:
'''
Now that the mdoel has been trained, I import a specific dataset which has one one key skill selected at a time, 
then I use the model to predict the salary given these particulars. Once I have the value, I deducted the median value
in order to find the increase as a function of the key skill.
'''

tester = pd.read_csv('./test2 - Sheet1.csv')


In [25]:
tester

,agile,cloud,sql,python,excel,aws
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,0,0,0,1,0,0
5,0,0,0,0,1,0
6,0,0,0,0,0,1


In [26]:
y_pred_list = []

In [27]:
y_pred = model2.predict(tester)
y_pred_list.append(y_pred)

In [28]:
y_pred_list

[array([161967.72104674, 215456.9328381 , 138106.38180663, 170586.03982972,
        291465.2817141 , 127190.16181145, 149390.59196784])]

In [29]:
y_pred = pd.DataFrame(y_pred_list)

In [30]:
y_pred = y_pred.T

In [31]:
tester.join(y_pred)

,agile,cloud,sql,python,excel,aws,0
0,0,0,0,0,0,0,161967.721047
1,1,0,0,0,0,0,215456.932838
2,0,1,0,0,0,0,138106.381807
3,0,0,1,0,0,0,170586.039830
4,0,0,0,1,0,0,291465.281714
5,0,0,0,0,1,0,127190.161811
6,0,0,0,0,0,1,149390.591968
